# Keras auto-compressor

This notebook was designed as an example to test the `keras_autocompressor` 
python module.

In [ ]:
# Import basic modules
import tensorflow as tf
import keras_tuner as kt
import tensorflow_datasets as tfds

# Allow memory grwoth in the docker container
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
# Change current directory to load our custom module
%cd /app/

In [ ]:
# Import the hyper model for the auto compression task
from auto_compressor.hypermodels import HyperCompressedMobileNetV2

In [ ]:
# Experiment settings
tensorflow_dataset = 'horses_or_humans'
batch_size = 16
max_search_epochs = 5

In [ ]:
# Load a dataset for training and testing
(ds_train, ds_test), ds_info = tfds.load(
    tensorflow_dataset, 
    split=['train', 'test'], 
    shuffle_files=True, 
    with_info=True, 
    as_supervised=True
)

In [ ]:
fig = tfds.show_examples(ds_train, ds_info)

In [ ]:
# Generate pre-processing function for the images
def preprocess_images(
        image, 
        label, 
        num_classes=ds_info.features['label'].num_classes
    ):
    # Resize images
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, (224,224))

    # Preprocess with the MobileNet function
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)

    # Change labels to categorical
    label = tf.cast(
        tf.one_hot(tf.cast(label, tf.int32), num_classes), dtype=tf.float32
    )
    
    return image, label

In [ ]:
# Pre-process the images
ds_train = ds_train.map(preprocess_images, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.map(preprocess_images, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
# Create batches for inference in both subsets
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(batch_size)   

ds_test = ds_test.batch(batch_size)

In [ ]:
# Create our Hyper model
hyper_model = HyperCompressedMobileNetV2(
    max_parameters=2309581,
    num_classes=ds_info.features['label'].num_classes,
    tau=0.8,
)

# Create the tuner object for our search
mobilenetv2_compressor = kt.Hyperband(
    hyper_model,
    max_epochs=max_search_epochs,
    objective=kt.Objective("val_acc_comp", direction="max"),
    directory='./logs/mobilenetv2/',
    project_name=tensorflow_dataset,
    overwrite=True
)

In [ ]:
# Run the hyperparameters search + auto compression
mobilenetv2_compressor.search(ds_train, validation_data=ds_test)

In [ ]:
# Get the best hyper parameters after the search
best_hyperparameters = mobilenetv2_compressor.get_best_hyperparameters()[0]
for key, item in best_hyperparameters.values.items():
    print(f'Hyperparameter: {key:20} | Value: {item}')

In [ ]:
# Get the best model
best_model = mobilenetv2_compressor.get_best_models()[0]

In [ ]:
# Compute the performance for the top-5 models obtained
for idx, sub_model in enumerate(mobilenetv2_compressor.get_best_models(5), start=1):
    metrics = sub_model.evaluate(ds_test, verbose=0)
    print(f'Top-{idx} model | val_accuracy: {metrics[1]:0.4f}  | params:' \
          + f' {sub_model.count_params()}')

In [ ]:
# Display the top-3 best models and their hyperparameters within the search
mobilenetv2_compressor.results_summary(num_trials=3)